In [1]:
from vllm import LLM, SamplingParams
from datasets import load_dataset

In [2]:
#model_id = "Qwen/Qwen2.5-0.5B"
model_id = "amang1802/mathllama_400M"
#model_id = "meta-llama/Llama-3.2-1B"

In [3]:
llm = LLM(model_id, max_model_len=2048, dtype="auto")

INFO 02-08 23:40:46 config.py:2167] Downcasting torch.float32 to torch.float16.
INFO 02-08 23:40:49 config.py:478] This model supports multiple tasks: {'classify', 'generate', 'embed', 'reward', 'score'}. Defaulting to 'generate'.
INFO 02-08 23:40:49 llm_engine.py:249] Initializing an LLM engine (v0.6.5) with config: model='amang1802/mathllama_400M', speculative_config=None, tokenizer='amang1802/mathllama_400M', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_tim

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-08 23:40:52 model_runner.py:1097] Loading model weights took 0.7628 GB
INFO 02-08 23:40:52 worker.py:241] Memory profiling takes 0.47 seconds
INFO 02-08 23:40:52 worker.py:241] the current vLLM instance can use total_gpu_memory (23.69GiB) x gpu_memory_utilization (0.90) = 21.32GiB
INFO 02-08 23:40:52 worker.py:241] model weights take 0.76GiB; non_torch_memory takes 0.11GiB; PyTorch activation peak memory takes 0.46GiB; the rest of the memory reserved for KV Cache is 19.99GiB.
INFO 02-08 23:40:52 gpu_executor.py:76] # GPU blocks: 32751, # CPU blocks: 6553
INFO 02-08 23:40:52 gpu_executor.py:80] Maximum concurrency for 2048 tokens per request: 255.87x
INFO 02-08 23:40:54 model_runner.py:1413] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-08 23:40:54 model_runner.py:1417] If out-of-memory error occurs during cudagraph captu

In [4]:
instructions = """
You are an AI assistant that helps people find information. User will you give you a question. Your task is to answer as faithfully as you can. While answering think step-by-step and justify your answer.
""".strip()

few_shot = """
Problem: James writes a 5-page letter to 3 different friends twice a week. How many pages does he write a year?

Solution: We should first calculate the number of pages James writes in a week. From that, calculating how many pages James write in a year is easy, because we know there are 52 weeks in a year.

In one week, James writes 5 x 3 x 2 = 30 pages of letters.
So, in one year, James writes 30 x 52 = 1560 pages of letters.

Answer: 1560

Problem: In a contest, each day they are handing out some prizes. On the first day, they handed out one prize. On the second day, they handed two prizes. Every day, the number of prizes they hand out increases by 1. The contest lasts a week. How many prizes did they hand out?

Solution: The total number of prizes is the sum of prizes they hand out each day. Since the contest lasts a week, this sum is

Total number of prizes = 1 + 2 + 3 + 4 + 5 + 6 + 7

We can use the formula for the sum of first n numbers to calculate this.

Total number of prizes = 7 * (7 + 1) / 2 = 7 * 8 / 2 = 7 * 4 = 28

Answer: 28
""".strip()
def solve(problems, attempt):
    prompts = [f"{instructions}\n\n{few_shot}\n\nProblem: {problem}\n\nSolution:" for problem in problems]
    outputs = llm.generate(prompts, SamplingParams(temperature=0.5, top_p=0.9, max_tokens=256, stop=["Problem:"]))
    return {f"{model_id}--{attempt}": [output.outputs[0].text for output in outputs]}

In [5]:
for ds_id in ["amang1802/math-vibe-gsm-similar", "amang1802/math-vibe-new"]:
    ds = load_dataset(ds_id)
    for attempt in range(3):
        ds = ds.map(lambda problems: solve(problems, attempt), batched=True, batch_size=5, input_columns=["problem"])
    ds['train'].push_to_hub(ds_id)

README.md:   0%|          | 0.00/580 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

Parameter 'function'=<function <lambda> at 0x7f342c15df80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

...


Processed prompts: 100%|██████████| 5/5 [00:00<00:00,  5.34it/s, est. speed input: 2398.80 toks/s, output: 948.40 toks/s]


Map:   0%|          | 0/5 [00:00<?, ? examples/s]


Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  4.22it/s, est. speed input: 1895.79 toks/s, output: 617.01 toks/s]


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]